In [1]:
%load_ext Cython
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import random
import codecs
from datetime import datetime
from hashlib import sha512
import itertools
import math

In [ ]:
#chop.py: simply chops the first however many lines
import codecs

OUTPUT_LINES = 1000

with codecs.open("data_v1.txt", "r", "utf-8") as infile:
    with codecs.open("data_out_{}.txt".format(OUTPUT_LINES), "w", "utf-8") as outfile:
        for i, line in enumerate(infile):
            if i > OUTPUT_LINES:
                break
            outfile.write(line)

In [ ]:
# randomsample.py
# Randomly samples however many lines needed 
import codecs
import random

TOTAL_LINES = 10**6 + 2
OUTPUT_LINES = 100000
p = OUTPUT_LINES/TOTAL_LINES

print("getting random sample of size {}".format(OUTPUT_LINES))
with codecs.open("data_v1.txt", "r", "utf-8") as infile:
	with codecs.open("data_random_{}.txt".format(OUTPUT_LINES), "w", "utf-8") as outfile:
		output_count = 0
		for i, line in enumerate(infile):
			if output_count < OUTPUT_LINES and random.random() < p:
				outfile.write(line)
				output_count += 1
			if output_count >= OUTPUT_LINES:
				break
print("done")

In [5]:
# findbr.py
# Finds (b,r) parameters required for documents of 
# JS similarity s to be in candidate groups with probability p

p = 0.98
q = 1 - p
s = 0.75
rrange = range(2, 20)
brange = range(0, 250, 5)

res = []

for b in brange:
    b = 10**(-b/100)
    for r in rrange:
        if (1 - s**r < q**b):
            res.append((int(1/b),r))
l = len(res)
print(l)
if l < 250:
    print(res)

86
[(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (2, 0), (2, 0), (2, 0), (3, 0), (3, 0), (3, 0), (4, 0), (5, 0), (5, 0), (6, 0), (7, 0), (7, 0), (8, 0), (10, 0), (11, 0), (12, 0), (12, 1), (14, 0), (14, 1), (15, 0), (15, 1), (17, 0), (17, 1), (19, 0), (19, 1), (22, 0), (22, 1), (25, 0), (25, 1), (28, 0), (28, 1), (31, 0), (31, 1), (35, 0), (35, 1), (39, 0), (39, 1), (44, 0), (44, 1), (50, 0), (50, 1), (56, 0), (56, 1), (63, 0), (63, 1), (70, 0), (70, 1), (79, 0), (79, 1), (89, 0), (89, 1), (100, 0), (100, 1), (112, 0), (112, 1), (125, 0), (125, 1), (125, 2), (141, 0), (141, 1), (141, 2), (158, 0), (158, 1), (158, 2), (177, 0), (177, 1), (177, 2), (199, 0), (199, 1), (199, 2), (223, 0), (223, 1), (223, 2), (251, 0), (251, 1), (251, 2), (281, 0), (281, 1), (281, 2)]


In [ ]:
# findthreshold.py
# for given (b,r) parameters, finds at which point s in [0,1] the slope is maximum.
# also finds probability of document pairs that have JS=S is in candidate pairs

srange = range(0,1000)
# ordered b first, r second
br = [(32,4), (16,8), (32,8), (8,4), (4,4), (8,2)]

fx = lambda s,b,r: 1-(1-s**r)**b
tx = lambda b,r: (1/b)**(1/r)
S = 0.75

for (b,r) in br:
    max_slope_threshold = (0, -1) # (s, f'(s))
    for s in srange:
        s = s/1000
        fpx = r*b*(1-s**r)**(b-1)*s**(r-1)
        if fpx > max_slope_threshold[1]:
            # print(max_slope_threshold)
            max_slope_threshold = (s, fpx)
    print("b {} r {} threshold {} p({}) {}".format(b, r, max_slope_threshold[0], S, fx(S, b, r)))



In [ ]:
# Finding optimal values of r and b for target JS = 0.75
# these are ordered r first, b second
l = [(4,32),(8,16),(8,32),(4,8),(4,4),(2,8)]

fig = plt.figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')
for i, (r, b) in enumerate(l):
    X = np.arange(0, 1, 0.01)
    Y = list(map(lambda x: 1-(1-x**r)**b, X))
    plt.subplot(2,3,1+i)
    plt.plot(X, Y)
    plt.title('$f(x) = 1−(1−x^{{{}}})^{{{}}}$'.format(r, b))
    plt.xlabel('x')
    plt.ylabel('f(x)')
    x = 0.75
    X = r
    Y = b
    print(X*Y*((1-x**X)**(Y-1))*(x**(X-1)))
plt.tight_layout()
plt.show()

In [2]:
%%cython

import string
from datetime import datetime
from hashlib import sha512
import itertools

def getWords(line):
   words = line.strip().lower().split()
   return list( map(lambda w: w.strip(string.punctuation), words) )

#k-shingle based on words
def kShingleWord(k, line, removeFirstWord=True):
    words = getWords(line)
    idx = words.pop(0)
    result = [' '.join([words[i+j] for j in range(k)]) for i in range(len(words)-k)]
    return result if removeFirstWord else (idx, result)
    
#k-shingle based on letters
def kShingle(k, line, removeFirstWord=True):
    words = getWords(line)
    idx = words.pop(0)
    line = ''.join(words)
    result = [''.join([line[i+j] for j in range(k)]) for i in range(len(line)-k)]
    # if removeFirstWord, return list of elements like ['0075c1c5', '005cf4fb',..., '00ccde89']
    # if not removeFirstWord, return list of elements like ('6066', ['0075c1c5', '005cf4fb',..., '00ccde89'])
    return result if removeFirstWord else (idx, result)
    
    

st = "6060 SECOND LORD. That approaches apace. I would gladly have him see his PAROLLES. What the dèvil should move me to undertake the recovery cheek of two pile and a half, but his right cheek is worn bare. COUNTESS. To be young again, if we could, I will be a fool in LAFEU. He was excellent indeed, madam; the King very lately spoke Boys. He was my father; and he is thrice a villain that says such father. He that so generally is at all times good must of Half won is match well made; match, and well make it; PAROLLES. It is to be recovered. But that the merit of service is HELENA. 'Till I have no wife, I have nothing in France.' Here is a pur of Fortune's, sir, or of Fortune's cat, but not CLOWN. You must not think I am so simple but I know the devil not seem to understand him, unless some one among us, whom we FIRST LORD. I am heartily sorry that he'll be glad of this. His part o' th' isle. Then does he say he lent me a solemn leave. His lordship will next morning for France. The them whipt; or I would send them to th' Turk to make eunuchs of. PAROLLES. 'Five or six thousand horse' I said-I will say true- 'or But take the High'st to witness. Then, pray you, tell me: But do not speak to me. Lead me to my chamber. Exeunt "
print("k-shingle word", kShingleWord(5, st)[:10])
print("k-shingle letters", kShingle(5, st)[:20])

# Jaccard similarity from two lists/sets of strings
# if only given s1, assume s1 holds both of the two lists and unpack it
def jaccardSim(s1, s2=None):
    if s2 is None:
        s1, s2 = s1
    s1 = set(s1)
    s2 = set(s2)
    return len(s1 & s2) / len(s1 | s2)

#st1 is documents 6393 and 7378 combined, st2 is 6393
# similarity should be around 0.5 for both word-shingle and letter-shingle
st1 = "6393 wear the surplice of humility over the black gown of a big heart. away; know it before the report come. If there be breadth enough BERTRAM. Why, if you have a stomach, to't, monsieur. If you think train'd me like a peasant, obscuring and hiding from me all BERTRAM. I'll lend it thee, my dear, but have no power KING. If it were yours by none of all these ways, See at the end of this file: * CONTENT NOTE (added in 2017) * face; if your lordship be in't, as I believe you are, you must COUNTESS. With very much content, my lord; and I wish it happily CHARMIAN. Nay, if an oily palm be not a fruitful prognostication, I LAFEU. I will buy me a son-in-law in a fair, and toll for this. FIRST SOLDIER. Boskos vauvado. I understand thee, and can speak thy PAROLLES. O my good lord, you were the first that found me. in death, which commits some loving act upon her, she hath such a PAROLLES. By the hand of a soldier, I will undertake it. child at fifty, to whom Herod of Jewry may do homage. Find me to sad a passage 'tis!-whose skill was almost as great as his in death, which commits some loving act upon her, she hath such a CLOWN. Faith, sir, 'a has an English name; but his fisnomy is more itself, which could not be her office to say is come, was 7378 whilst I have a tooth in my head. Why, he's able to lead her a against his valour; and my state that way is dangerous, since I PAROLLES. The Duke knows him for no other but a poor officer of to th' Queen? O that I knew this husband, which you say must ANTONY. That which is now a horse, even with a thought Fare you well, my lord; and believe this of me: there can be no beat thee. I think thou wast created for men to breathe ANTONY. Do so, we'll speak to them; and to-night I'll force much of my father in me as you, albeit I confess your coming dare not give. Wherefore, what's the instance? Tongue, I must put the toothpick, which wear not now. Your date is better in your ANTONY. I would they'd fight i' th' fire or i' th' air; PAROLLES. I know not what the success will be, my lord, but the neighbouring languages, therefore we must every one be a man of KING. Know you this ring? This ring was his of late. when old robes are worn out there are members to make new. If ENOBARBUS. Why, sir, give the gods a thankful sacrifice. When it sadness. My brother Jaques he keeps at school, and report speaks PAROLLES. I beseech your honour to hear me one single word. when he number'd thirty; 'a will be here to-morrow, or I am "
st2 = "6393 wear the surplice of humility over the black gown of a big heart. away; know it before the report come. If there be breadth enough BERTRAM. Why, if you have a stomach, to't, monsieur. If you think train'd me like a peasant, obscuring and hiding from me all BERTRAM. I'll lend it thee, my dear, but have no power KING. If it were yours by none of all these ways, See at the end of this file: * CONTENT NOTE (added in 2017) * face; if your lordship be in't, as I believe you are, you must COUNTESS. With very much content, my lord; and I wish it happily CHARMIAN. Nay, if an oily palm be not a fruitful prognostication, I LAFEU. I will buy me a son-in-law in a fair, and toll for this. FIRST SOLDIER. Boskos vauvado. I understand thee, and can speak thy PAROLLES. O my good lord, you were the first that found me. in death, which commits some loving act upon her, she hath such a PAROLLES. By the hand of a soldier, I will undertake it. child at fifty, to whom Herod of Jewry may do homage. Find me to sad a passage 'tis!-whose skill was almost as great as his in death, which commits some loving act upon her, she hath such a CLOWN. Faith, sir, 'a has an English name; but his fisnomy is more itself, which could not be her office to say is come, was "
s1 = kShingle(5, st1)
s2 = kShingle(5, st2)
print("JS based on k-word-shingles", jaccardSim(s1, s2))
s1 = kShingleWord(5, st1)
s2 = kShingleWord(5, st2)
print("JS based on k-letter-shingles", jaccardSim(s1, s2))


# shingles should be given as list of strings
# this hashes all shingles (8 hash functions) and for each hash function find minimum value hashes.
# the hash might not be a number but okay if hash is consistent and there is a clear ordering (alphabetical here)
def minhash(shingles):
    h = 16 # no. of hashes we want
    l = 8 # length of each hash
    hashes = ['z'*l] * h # minhashes are initialized to 'zzzzzzzz'
    m = sha512()
    for shingle in shingles:
        m.update(shingle.encode('utf-8'))
        newhash = m.hexdigest()
        newhashes = [newhash[i:i+l] for i in range(0,l*h,l)] #divide 128 characters into 16 hashes of length 8
        hashes = [min(hashes[i], newhashes[i]) for i in range(h)]
    return hashes

# st2 = "6393 wear the surplice of humility over the black gown of a big heart. away; know it before the report come. If there be breadth enough BERTRAM. Why, if you have a stomach, to't, monsieur. If you think train'd me like a peasant, obscuring and hiding from me all BERTRAM. I'll lend it thee, my dear, but have no power KING. If it were yours by none of all these ways, See at the end of this file: * CONTENT NOTE (added in 2017) * face; if your lordship be in't, as I believe you are, you must COUNTESS. With very much content, my lord; and I wish it happily CHARMIAN. Nay, if an oily palm be not a fruitful prognostication, I LAFEU. I will buy me a son-in-law in a fair, and toll for this. FIRST SOLDIER. Boskos vauvado. I understand thee, and can speak thy PAROLLES. O my good lord, you were the first that found me. in death, which commits some loving act upon her, she hath such a PAROLLES. By the hand of a soldier, I will undertake it. child at fifty, to whom Herod of Jewry may do homage. Find me to sad a passage 'tis!-whose skill was almost as great as his in death, which commits some loving act upon her, she hath such a CLOWN. Faith, sir, 'a has an English name; but his fisnomy is more itself, which could not be her office to say is come, was "

startingTime = datetime.now()
print("minhashes of a single document", minhash(kShingle(5, st2)))
print("minhashes of a single document", minhash(kShingle(5, st1)))
print(datetime.now()-startingTime)

def C(lst, r):
    #returns all "choose r" combinations as a set
    return set(map(lambda x: tuple(sorted(x)), itertools.combinations(lst, r)))


k-shingle word ['second lord that approaches apace', 'lord that approaches apace i', 'that approaches apace i would', 'approaches apace i would gladly', 'apace i would gladly have', 'i would gladly have him', 'would gladly have him see', 'gladly have him see his', 'have him see his parolles', 'him see his parolles what']
k-shingle letters ['secon', 'econd', 'condl', 'ondlo', 'ndlor', 'dlord', 'lordt', 'ordth', 'rdtha', 'dthat', 'thata', 'hatap', 'atapp', 'tappr', 'appro', 'pproa', 'proac', 'roach', 'oache', 'aches']
JS based on k-word-shingles 0.5052325581395349
JS based on k-letter-shingles 0.4883720930232558
minhashes of a single document ['00262861', '0004bbc1', '00322021', '001ab8cf', '00952c4a', '00515c08', '0053f03a', '00451141', '0015baa9', '000c519f', '001fa1aa', '002b14ff', '00122380', '0061ab54', '00128b05', '00419964']
minhashes of a single document ['00262861', '0004bbc1', '00213738', '001ab8cf', '000fc8cf', '000d7127', '0053f03a', '00451141', '000993e8', '000c519f', '001fa

In [ ]:
#plot histogram of distribution of JS
# usually between 0.0 and 0.2 (99.99%)
OUTPUT_LINES = 1000
k = 5

with codecs.open("data_random_{}.txt".format(OUTPUT_LINES), "r", "utf-8") as infile:
    lst = map(lambda line: kShingleWord(k, line), infile.read().splitlines()) # samples

startTime = datetime.now()
print(startTime)
pairs = list(itertools.combinations(lst, 2)) # all document combinations
jaccard_similarities = list(map(jaccardSim, pairs))

plt.subplot()
plt.title("Jaccard Similarities for #documents={}".format(OUTPUT_LINES))
plt.hist(jaccard_similarities,'auto')
plt.show()
print(datetime.now() - startTime)

In [ ]:
# What are the lowest and highest values of JS in the random file?
js = sorted(jaccard_similarities)
js[0], js[-1]

In [ ]:
# SHA-512 api example
m = sha512()
m.update("hello world".encode('utf-8'))
print(m.hexdigest(), len(m.hexdigest()))


In [ ]:
# How long does it take to find shingles? Less than 10 seconds for 10^6 documents
OUTPUT_LINES = 100
k = 5

startingTime = datetime.now()
with codecs.open("data_random_{}.txt".format(OUTPUT_LINES), "r", "utf-8") as infile:
    lst = map(lambda line: kShingle(5, line), infile.read().splitlines()) # samples

print(datetime.now()-startingTime)

In [ ]:
# Caculate minhashes and save in file
# takes about 5.6 ms for running minhash with 16 hashes (all derived from 1 SHA-512 function call) on each document.
# expected to take about 1.5 hrs for 10^6 documents

k = 5

startingTime = datetime.now()
print("Started at {}".format(startingTime))

with codecs.open("data_v1.txt", "r", "utf-8") as infile:
    lst = map(lambda line: kShingle(k, line, False), infile.read().splitlines()) # samples

print("Finished shingling data at {}".format(datetime.now()))

with codecs.open("minhash_data_v1.txt", "w", "utf-8") as outfile:
    for (idx, shingles) in lst:
        outfile.write("{} {}\n".format(idx, ' '.join(minhash(shingles))))

print("Finished minhashing data at {}".format(datetime.now()-startingTime))


In [3]:
# Put documents into buckets
# create 16 dictionaries, one for each hash.
# for each hashed document, put the index of the document into the 16 dictionaries with the hash as the key. like so:

# h1dict = {ecba2d : [1]}
# h2dict = {dfqfe3 : [1]}
# etc.

startingTime = datetime.now()
print(startingTime)
with codecs.open("minhash_data_v1.txt", "r") as infile:
    lst = map(getWords, infile.read().splitlines())
    

h = 16 # number of hash functions
buckets = [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
print(buckets)
for (j, words) in enumerate(lst):
    idx = int(words[0]) #idx of doc
    hashes = words[1:] #16 hashes
    for (i, hashVal) in enumerate(hashes): # put each hash into their respective buckets
#         bucket = buckets[(i+j)%h].get(hashVal, [])
        bucket = buckets[i].get(hashVal, [])
        bucket.append(idx)
        buckets[i][hashVal] = bucket
#         if j < 5:
#             print('i', i, " idx", idx, " bucket", bucket, " buckets[i]", buckets[i], " hashVal", hashVal)
print("length of buckets",len(buckets))
print(datetime.now() - startingTime)
print(list(buckets[1].items())[:30])

2018-02-18 02:50:32.786674
[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
length of buckets 16
0:00:36.517971
[('0052f3a0', [1]), ('0070a6be', [2]), ('001f135a', [3]), ('0013811e', [4]), ('000c0342', [5]), ('00700253', [6]), ('0016c791', [7]), ('01312c26', [8]), ('00926889', [9]), ('00051d28', [10]), ('00230c45', [11]), ('01d331f2', [12]), ('00e47b0e', [13]), ('0010c66f', [14]), ('008650f4', [15]), ('000f629a', [16]), ('00703ae1', [17]), ('006351b0', [18]), ('002fe2e1', [19, 146535]), ('003922a8', [20]), ('0031c1fc', [21]), ('0001c729', [22]), ('005c00f7', [23]), ('0022bae1', [24]), ('0036299c', [25]), ('002a5e2f', [26]), ('0006ea45', [27]), ('005a02f6', [28]), ('00317848', [29]), ('001a71ea', [30, 768630, 782122])]


In [ ]:
# [len(buckets[i]) for i in range(16)]
# print(list(buckets[0].values())[:1000])
print(list(buckets[1].values())[:1000])
print(len(buckets))

In [8]:
# divide into b=8 bands, r=2 rows
# for each band, find all combinations of documents that are put in same buckets in all rows

b=8
r=2
candidates = set([])
keylessBuckets = list(map(lambda row: set(map(tuple, row.values())), buckets)) #list of set of tuples

startingTime = datetime.now()
print(startingTime)

for i in range(0,b*r,r):
    print(i, i+r)
    band = keylessBuckets[i:i+r]

    listOfRowCandidates = []
    for row in band: # row is [['1'], ['1', '18566'], ['1'], ['1'], ['1']..] set of tuples
        rowCandidates = set([]) # all pairs from a row
        for bucket in row: # bucket is ['2', '126908', '203926', '278946', '746802'] tuples
            rowCandidates |= C(bucket, 2)
        listOfRowCandidates.append(rowCandidates)
        print("Finished row", len(rowCandidates))
    intersection = listOfRowCandidates[0] # intersections of rows in same band. pair becomes candidate if exists in all rows of a band
    for rowCandidates in listOfRowCandidates:
        intersection &= rowCandidates
    candidates |= intersection
    print("length of intersections within band", len(intersection))
    print("length of candidate weekends", len(candidates))

print(datetime.now()-startingTime)
# print(len(candidates))

2018-02-18 03:19:04.613484
0 2
Finished row 11741895 0
Finished row 12187899 0
length of candidate weekends 83832
2 4
Finished row 13699087 83832
Finished row 130388013 83832
length of candidate weekends 1071232
4 6
Finished row 9966891 987408
Finished row 10250667 987408
length of candidate weekends 1205779
6 8
Finished row 10292661 134558
Finished row 14111571 134558
length of candidate weekends 1359942
8 10
Finished row 9471510 154174
Finished row 14132027 154174
length of candidate weekends 1562609
10 12
Finished row 11909920 202677
Finished row 42645344 202677
length of candidate weekends 1658155
12 14
Finished row 8714414 95714
Finished row 12851207 95714
length of candidate weekends 1851122
14 16
Finished row 12599113 193027
Finished row 15227033 193027
length of candidate weekends 2172651
0:07:56.926840


In [ ]:
print(list(keylessBuckets[0])[:30])
print(candidates)
print(len(candidates))

In [15]:
docsInCandidates = set([])
for (i,j) in candidates:
    docsInCandidates |= set([str(i),str(j)])
print(len(docsInCandidates))

30865


In [14]:
# for each candidate pair in the list, 
# calculate the actual Jaccard Similarity and if JS > 0.75, 
# add to output list
k = 5

startingTime = datetime.now()
print(startingTime)
with codecs.open("data_v1.txt", "r", "utf-8") as infile:
    lst = list(map(lambda line: kShingle(k, line), infile.read().splitlines())) # samples

print("done processing...", datetime.now()-startingTime)

    
output = []
count = 0
logcount = 0
for (i,j) in candidates:
    if (jaccardSim(lst[i+1], lst[j+1]) > 0.75):
#         print(lst[i+1],lst[j+1])
        output.append((i,j))
        count+=1
        if math.log10(count) == logcount:
            print(count)
            logcount += 1
print(len(output))

print(datetime.now() - startingTime)

2018-02-18 03:45:36.367299


KeyboardInterrupt: 

In [ ]:
# b=8
# r=2
# candidates = set([])
# startingTime = datetime.now()
# print(startingTime)
# keylessBuckets = list(map(lambda row: set(map(tuple, row.values())), buckets))

# print("finished keylessbucketing", datetime.now()-startingTime)

# for i in range(0,b,r):
#     band = keylessBuckets[i:i+r]
#     intersection = set([]) # intersections of rows in same band. pair becomes candidate if exists in all rows of a band
#     for row in band: # row is [['1'], ['1', '18566'], ['1'], ['1'], ['1']..]
#         rowCandidates = set([]) # all pairs from a row
#         print("length of row", len(row))
#         for i, bucket in enumerate(row): # bucket is ['2', '126908', '203926', '278946', '746802']
#             rowCandidates |= C(bucket, 2)
#             if i%(10**4):
#                 print("finished 10**4 buckets ",datetime.now()-startingTime)                
#         print("finished row", datetime.now()-startingTime)

In [ ]:
lst = ('df','asd','asdf')
for i, ls in enumerate(lst):
    print(i,ls)
print(C(lst,2))

In [12]:
count = 0
logcount = 0
for i in range(10**20):
    count+=1
    if math.log10(count) == logcount:
        print(count)
        logcount += 1

1
10
100
1000
10000
100000
1000000
